# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,-26.06,72,40,0.00,US,1675615498
1,1,kidero,42.1932,45.9614,-7.77,91,97,0.77,RU,1675615642
2,2,chuy,-33.6971,-53.4616,25.90,49,12,4.96,UY,1675615517
3,3,rikitea,-23.1203,-134.9692,25.88,75,8,7.17,PF,1675615473
4,4,jardim,-21.4803,-56.1381,32.10,45,37,1.12,BR,1675615645


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot


map_plot_hum = city_data_df.hvplot.points("Lng",
                                         "Lat",
                                         geo = True,
                                         tiles = "OSM",
                                         size = "Humidity",
                                         color = "City")
#py.plot.scatter()

# Display the map
map_plot_hum

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_data_df = city_data_df.loc[(city_data_df["Humidity"]<50)
                                       & (city_data_df["Max Temp"]>10) & (city_data_df["Max Temp"]<25)
                                       & (city_data_df["Cloudiness"]<30) & (city_data_df["Country"] != "US")]

# Drop any rows with null values
narrow_city_data_df = narrow_city_data_df.dropna()

# Display sample data
narrow_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,81,hindoria,23.9000,79.5667,18.70,32,0,1.97,IN,1675615678
101,101,ponta do sol,32.6667,-17.1000,17.85,48,29,1.72,PT,1675615691
141,141,char bhadrasan,23.3167,90.2167,18.82,38,4,1.51,BD,1675615720
146,146,montoro,38.0241,-4.3834,19.32,39,1,0.92,ES,1675615359
182,182,broken hill,-31.9500,141.4333,20.78,36,0,4.11,AU,1675615736
198,198,serta,39.8085,-8.0988,16.08,49,0,2.47,PT,1675615749
237,237,ladnun,27.6500,74.3833,21.52,26,0,3.47,IN,1675615781
270,270,rajapur,25.3833,81.1500,17.12,45,0,1.06,IN,1675615793
275,275,nalut,30.3333,10.8500,12.69,40,3,4.48,LY,1675615795
315,315,xingyi,24.1877,102.8170,10.12,48,6,1.69,CN,1675615836


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
81,hindoria,IN,23.9000,79.5667,32,
101,ponta do sol,PT,32.6667,-17.1000,48,
141,char bhadrasan,BD,23.3167,90.2167,38,
146,montoro,ES,38.0241,-4.3834,39,
182,broken hill,AU,-31.9500,141.4333,36,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel

radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey":geoapify_key,
          "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

 #Display sample data
hotel_df

Starting hotel search
hindoria - nearest hotel: No hotel found
ponta do sol - nearest hotel: Estalagem Ponta do Sol
char bhadrasan - nearest hotel: No hotel found
montoro - nearest hotel: Hostal Restaurante Montoro
broken hill - nearest hotel: Ibis Styles
serta - nearest hotel: Residence Inn
ladnun - nearest hotel: No hotel found
rajapur - nearest hotel: No hotel found
nalut - nearest hotel: No hotel found
xingyi - nearest hotel: 通印大酒店
providencia - nearest hotel: No hotel found
huejuquilla el alto - nearest hotel: No hotel found
mosquera - nearest hotel: Hotel San Martin
yulara - nearest hotel: Outback Pioneer Hotel & Lodge
caxias - nearest hotel: Hjem
mildura - nearest hotel: Chaffey Motor Inn
requena - nearest hotel: Hotel Avenida
aswan - nearest hotel: Yaseen hotel
christchurch - nearest hotel: Ibis Hotel Christchurch


,City,Country,Lat,Lng,Humidity,Hotel Name
81,hindoria,IN,23.9000,79.5667,32,No hotel found
101,ponta do sol,PT,32.6667,-17.1000,48,Estalagem Ponta do Sol
141,char bhadrasan,BD,23.3167,90.2167,38,No hotel found
146,montoro,ES,38.0241,-4.3834,39,Hostal Restaurante Montoro
182,broken hill,AU,-31.9500,141.4333,36,Ibis Styles
198,serta,PT,39.8085,-8.0988,49,Residence Inn
237,ladnun,IN,27.6500,74.3833,26,No hotel found
270,rajapur,IN,25.3833,81.1500,45,No hotel found
275,nalut,LY,30.3333,10.8500,40,No hotel found
315,xingyi,CN,24.1877,102.8170,48,通印大酒店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points("Lng",
                                         "Lat",
                                         geo = True,
                                         tiles = "OSM",
                                         size = "Humidity",
                                         color = "City",
                                         hover_cols = ["Hotel Name", "Country"])
# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)